In [1]:
import sys

sys.path.append("../")

from pae.analysis.scalar_v2 import HLFAnalysis
from pae.models.nn import PaeBuilder

import tensorflow as tf
import numpy as np
SEED = 42
np.random.seed(SEED) 
tf.random.set_seed(SEED)

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

2 Physical GPUs, 2 Logical GPUs


2022-05-16 02:57:32.703347: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-16 02:57:34.102365: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 30989 MB memory:  -> device: 0, name: NVIDIA Tesla V100S-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2022-05-16 02:57:34.104050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30989 MB memory:  -> device: 1, name: NVIDIA Tesla V100S-PCIE-32GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


In [3]:
from pae.utils import load_json, dump_json

from pae.loaders.LHCO import ScalarLoaderLHCO, DatasetBuilder

x = ScalarLoaderLHCO.from_json("../pae/configs/loader/bbox1_scalar_2j.json")
mjj = ScalarLoaderLHCO.from_json("../pae/configs/loader/bbox1_scalar_mjj.json")
builder = DatasetBuilder(x, mjj)
builder.data_preparation(sample_sizes ={'box_s':834, 'box_b': 999166, 'bkg': 500_000}, fit_key='bkg')
# dataset = builder.make_dataset(train = {'box_s':834, 'box_b': 999166}, test={'box_s':834, 'box_b': 999166}, replace=True)
dataset = builder.make_dataset(train = {'box_s':100, 'box_b': 1900}, test={'box_s':100, 'box_b': 1900}, replace=True)

In [4]:
import tensorflow.keras as tfk
import tensorflow as tf

from pae.models.autoencoder import DenseAutoencoder
from pae.models.flows import CondMAF, MAF
from pae.models.nn import PaeBuilder
from pae.analysis.scalar_v2 import HLFAnalysis

task = HLFAnalysis("../stuff.json", dataset.copy())

In [5]:
task.cross_validate()

2022-05-16 02:58:00.095173: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.



Epoch 17: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.

Epoch 49: ReduceLROnPlateau reducing learning rate to 0.0001999999862164259.
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.

Epoch 24: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.

Epoch 29: ReduceLROnPlateau reducing learning rate to 0.0001999999862164259.

Epoch 34: ReduceLROnPlateau reducing learning rate to 3.9999996079131965e-05.

Epoch 39: ReduceLROnPlateau reducing learning rate to 7.99999907030724e-06.

Epoch 44: ReduceLROnPlateau reducing learning rate to 1.599999814061448e-06.

Epoch 49: ReduceLROnPlateau reducing learning rate to 3.1999995826481613e-07.

Epoch 16: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.

Epoch 41: ReduceLROnPlateau reducing learning rate to 0.0001999999862164259.

Epoch 22: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.

Epoch 27: Reduce

In [6]:
dataset.keys()

dict_keys(['mjj_train', 'mjj_test', 'x_train', 'x_test', 'labels_train', 'labels_test'])

In [7]:
print(dataset["x_train"])

[[0.         0.03203203 0.48898899 ... 0.99647507 0.8385234  0.46896897]
 [0.         0.08658659 0.31331331 ... 0.99487758 0.69713664 0.27277277]
 [0.         0.26376376 0.71271271 ... 0.99442647 0.72039235 0.74574575]
 ...
 [0.         0.36836837 0.83083083 ... 0.71862961 0.60610817 0.78878879]
 [0.         0.26376376 0.56906907 ... 0.2747823  0.27068633 0.60910911]
 [0.         0.47497497 0.71271271 ... 0.97008277 0.71366363 0.71821822]]
